In [2]:
# !py -m pip install pycalphad
# !py -m pip install numpy
# !py -m pip install tqdm
# !py -m pip install ipywidgets

In [ ]:
from pycalphad import Database, equilibrium, variables as v
# from pycalphad.core.utils import filter_phases
import random
import numpy as np
import time
from tqdm import tqdm
# from tqdm.auto import tqdm

# constants:
P = 101325
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)
# TODO: check 'PM' param
all_components = ['LI', 'BE', 'NA', 'MG', 'AL', 'K', 'CA', 'SC', 'TI', 'V', 'CR', 'MN', 'FE',
                  'CO', 'NI', 'CU', 'ZN', 'GA', 'Y', 'ZR', 'NB', 'MO', 'TC',
                  'RH', 'PD', 'AG', 'CD', 'IN', 'SN', 'BA', 'LA', 'CE', 'PR', 'ND', 'SM', 'EU',
                  'GD', 'TB', 'DY', 'HO', 'ER', 'HF', 'TA', 'W', 'RE', 'OS', 'IR',
                  'PT', 'AU', 'HG', 'TL', 'PB', 'BI', 'C', 'SI']
all_phases = list(db.phases)
all_phases.sort()
all_components.sort()

In [48]:
def getPhases(int_mask, all_phases=all_phases):
    return [all_phases[i] for i in range(len(all_phases)) if (int(int_mask) & (1 << i)) > 0]

def phasesToMask(phases, all_phases=all_phases):
    phases.sort()
    mask = [1 if phase in phases else 0 for phase in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)


def phaseToMask(phase, all_phases=all_phases):
    mask = [1 if phase == phase_ else 0 for phase_ in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)

def compToMask(components, all_components=all_components):
    components.sort()
    mask = [1 if comp in components else 0 for comp in all_components][::-1]
    return int(''.join(map(str, mask)), 2)

def getComponents(int_mask, all_components=all_components):
    return [all_components[i] for i in range(len(all_components)) if (int_mask & (1 << i)) > 0]

def parseArray(amounts_str):
    amounts = amounts_str.replace('[', '').replace(']', '')
    return np.array(amounts.split(','), dtype=np.float32)


In [60]:
import pandas as pd

df = pd.read_csv('DataSets/equilibrium_result.csv',delimiter = ';')
df = df[df['Error'].isna()].reset_index(drop=True)
df = df.drop(['Error'], axis =1)
df['Components'] = df['Components'].apply(getComponents)
df['phases'] = df['phases'].apply(getPhases)
df['possible_phases'] = df['possible_phases'].apply(getPhases)
df['amounts'] = df['amounts'].apply(parseArray)
print(df.dtypes)
df

iter                 int64
T                    int64
amounts             object
Components          object
phases              object
ellapsed_time      float64
possible_phases     object
P                    int64
dtype: object


,iter,T,amounts,Components,phases,ellapsed_time,possible_phases,P
0,0,430,"[0.08967187, 0.1772739, 0.27985972, 0.05597027...","[BI, C, MG, OS, PR, SM, TC, ZR]","[BI2MG3_D52, FCC_A1, GRAPHITE, HCP_A3]",13.438566,"[ALLI, ALMG_GAMMA1, BCC_A2, BCT_A5, BI2MG3_BET...",101325
1,1,2713,"[0.21780217, 0.42648143, 0.09295164, 0.0521797...","[BI, C, MG, OS, PR, SM, TC, ZR]","[FCC_A1, GAS, GRAPHITE, LIQUID]",13.299496,"[ALLI, ALMG_GAMMA1, BCC_A2, BCT_A5, BI2MG3_BET...",101325
2,2,718,"[0.105666816, 0.1026478, 0.46671695, 0.0952720...","[BI, C, MG, OS, PR, SM, TC, ZR]","[BI2MG3_D52, FCC_A1, GRAPHITE, HCP_A3, MG2SM_C...",13.746437,"[ALLI, ALMG_GAMMA1, BCC_A2, BCT_A5, BI2MG3_BET...",101325
3,0,430,"[0.08967187, 0.1772739, 0.27985972, 0.05597027...","[BI, C, MG, OS, PR, SM, TC, ZR]","[BI2MG3_D52, FCC_A1, GRAPHITE, HCP_A3]",13.784137,"[ALLI, ALMG_GAMMA1, BCC_A2, BCT_A5, BI2MG3_BET...",101325
4,1,2713,"[0.21780217, 0.42648143, 0.09295164, 0.0521797...","[BI, C, MG, OS, PR, SM, TC, ZR]","[FCC_A1, GAS, GRAPHITE, LIQUID]",13.638592,"[ALLI, ALMG_GAMMA1, BCC_A2, BCT_A5, BI2MG3_BET...",101325
...,...,...,...,...,...,...,...,...
2478,20,863,"[0.32288572, 0.29498613, 0.12984245, 0.0516073...","[AU, CD, CR, FE, GD, HG, LA, PB]","[AU6LA, CD58GD13, GD6PB7, LIQUID, SIGMA]",25.770236,"[AGCD_ETA, AGSB_ORTHO, AL5FE4, ALCR2_C11B, AU2...",101325
2479,21,2909,"[0.09848462, 0.1693075, 0.18832855, 0.1981198,...","[AU, CD, CR, FE, GD, HG, LA, PB]","[AUPB3, LIQUID]",27.020861,"[AGCD_ETA, AGSB_ORTHO, AL5FE4, ALCR2_C11B, AU2...",101325
2480,22,827,"[0.11150886, 0.20775788, 0.21381086, 0.1547906...","[AU, CD, CR, FE, GD, HG, LA, PB]","[AU2LA, AULA_B27, CD3GD_D019, GD6PB7, LIQUID, ...",26.539249,"[AGCD_ETA, AGSB_ORTHO, AL5FE4, ALCR2_C11B, AU2...",101325
2481,23,2438,"[0.06484277, 0.25925854, 0.3462349, 0.06592706...","[AU, CD, CR, FE, GD, HG, LA, PB]","[AUPB3, LIQUID]",29.425763,"[AGCD_ETA, AGSB_ORTHO, AL5FE4, ALCR2_C11B, AU2...",101325


In [61]:
df_row = df.iloc[0]
print(df_row)

T = df_row['T']
amounts = df_row['amounts']
Components = df_row['Components']
phases = df_row['phases']
ellapsed_time = df_row['ellapsed_time']
possible_phases = df_row['possible_phases']
phases

iter                                                               0
T                                                                430
amounts            [0.08967187, 0.1772739, 0.27985972, 0.05597027...
Components                           [BI, C, MG, OS, PR, SM, TC, ZR]
phases                        [BI2MG3_D52, FCC_A1, GRAPHITE, HCP_A3]
ellapsed_time                                              13.438566
possible_phases    [ALLI, ALMG_GAMMA1, BCC_A2, BCT_A5, BI2MG3_BET...
P                                                             101325
Name: 0, dtype: object


['BI2MG3_D52', 'FCC_A1', 'GRAPHITE', 'HCP_A3']

In [63]:
import pandas as pd
import numpy as np

def sortTColumns(columns):
    t_columns = [col for col in df_f.columns if col[0] == 'T' and col[1] != '_']
    t_log_columns = [col for col in t_columns if 'log' in col]
    t_columns = set(t_columns) - set(t_log_columns)
    minus_t = [col for col in t_columns if '-' in col]
    minus_t.sort(reverse=True)
    not_minus_t = list(set(t_columns) - set(minus_t))
    not_minus_t.sort()
    return minus_t + not_minus_t, t_log_columns

df_f = pd.read_csv('ParsedData/phases_functions.csv',delimiter = ',', low_memory=False)
max_global_t = 6000
max_t = 3000

df_f.rename(columns={"T": "T**1.0"}, inplace=True)
df_f.drop(['Unnamed: 0'], axis =1, inplace=True)
df_f = df_f.dropna(subset=['T_min']).reset_index(drop = True)
df_f['T_max'].fillna(value=max_global_t, inplace=True)
df_f.loc[df_f['T_max']> max_t , 'T_max'] = max_t


# t_columns.sort()
# t_columns.sort(reverse=True)
t_columns, t_log_columns = sortTColumns(df_f.columns)
print(len(t_columns))
print(t_log_columns)
# print(t_columns)
# 

s_columns = [col for col in df_f.columns if col[0] == 'S']
print(s_columns)

df_f[s_columns] = df_f[s_columns].replace("VA", np.nan)

all_columns = [col for col in df_f.columns if col not in t_columns]
df_f[all_columns]
# df['Tlog(1.0e-5*P)'].notna().sum()

12
['Tlog(T)', 'Tlog(1.0e-5*P)']
['S1', 'S2', 'S3', 'S4', 'S5']


,T_min,T_max,Tlog(T),phase,S1,S2,S3,S4,S5,Tlog(1.0e-5*P)
0,298.15,1873.000,-27.744000,A15_CR3PT,CR,PT,NaN,NaN,NaN,NaN
1,298.15,700.000,-24.367198,A2_BCC,AL,NaN,NaN,NaN,NaN,NaN
2,298.15,2900.000,NaN,A2_BCC,AL,NaN,NaN,NaN,NaN,NaN
3,700.00,933.473,-38.584430,A2_BCC,AL,NaN,NaN,NaN,NaN,NaN
4,298.15,3000.000,NaN,A2_BCC,AL,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
23643,3000.00,3000.000,-31.400000,ZRB_B1,ZR,B,NaN,NaN,NaN,NaN
23644,2128.00,3000.000,-42.144000,ZRB_B1,ZR,B,NaN,NaN,NaN,NaN
23645,298.15,3000.000,-73.210250,ZRPB2,ZR1PB2,NaN,NaN,NaN,NaN,NaN
23646,298.15,3000.000,-45.186310,ZRSI,SI,ZR,NaN,NaN,NaN,NaN


In [66]:
phase = possible_phases[0]
phase

'ALLI'

In [72]:
df_short = df_f[df_f['phase']==phase]
df_short = df_short[(T > df_short['T_min']) & (T <= df_short['T_max'])]
df_short

,T_min,T_max,T**0,T**(-1.0),T**1.0,T**2.0,T**3.0,Tlog(T),phase,S1,...,T**(-9.0),S3,S4,T**(-3.0),T**(-2.0),T**7.0,S5,T**(-5.0),T**(-7.0),Tlog(1.0e-5*P)
1324,200.00,453.6,-21067.634,319988.0,435.274964,0.070934,-3.973963e-05,-77.880976,ALLI,LI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1327,298.15,923.0,-10534.680,157900.0,287.351094,0.000972,-2.787338e-06,-52.369956,ALLI,MG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1328,298.15,3000.0,0.000,NaN,NaN,NaN,NaN,NaN,ALLI,MG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1329,298.15,923.0,41632.660,78950.0,143.675547,0.000486,-1.393669e-06,-26.184978,ALLI,MG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1331,298.15,952.0,-57429.807,204480.0,269.644280,-0.020165,5.504833e-06,-44.405000,ALLI,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1333,298.15,700.0,16023.850,74092.0,137.093000,-0.001885,-8.776640e-07,-24.367198,ALLI,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1336,298.15,453.6,39416.184,159994.0,217.637440,0.035467,-1.986982e-05,-38.940488,ALLI,LI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1339,298.15,3000.0,20000.000,NaN,NaN,NaN,NaN,NaN,ALLI,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1340,298.15,3000.0,-26000.000,NaN,NaN,NaN,NaN,NaN,ALLI,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1341,298.15,3000.0,2000.000,NaN,NaN,NaN,NaN,NaN,ALLI,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
# comp = Components[0]
# df_final = df_short.loc[(df_short['S1'] == comp) | (df_short['S2'] == comp) | (df_short['S3'] == comp) | (df_short['S4'] == comp) | (df_short['S5'] == comp)]
comp = Components[2]
print(comp)
df_final = df_short.loc[(df_short['S1'] == comp) | (df_short['S2'] == comp) | (df_short['S3'] == comp) | (df_short['S4'] == comp) | (df_short['S5'] == comp)]
df_final[all_columns]

MG


,T_min,T_max,Tlog(T),phase,S1,S2,S3,S4,S5,Tlog(1.0e-5*P)
1327,298.15,923.000,-52.369956,ALLI,MG,MG,NaN,NaN,NaN,NaN
1328,298.15,3000.000,NaN,ALLI,MG,MG,NaN,NaN,NaN,NaN
1329,298.15,923.000,-26.184978,ALLI,MG,NaN,NaN,NaN,NaN,NaN
1330,923.00,3000.000,-34.308800,ALLI,MG,NaN,NaN,NaN,NaN,NaN
1344,298.15,923.000,-26.184978,ALLI,AL,MG,NaN,NaN,NaN,NaN
1345,298.15,3000.000,NaN,ALLI,AL,MG,NaN,NaN,NaN,NaN
1346,700.00,933.473,-38.584430,ALLI,AL,MG,NaN,NaN,NaN,NaN
1347,298.15,3000.000,NaN,ALLI,AL,MG,NaN,NaN,NaN,NaN
1348,200.00,453.600,-38.940488,ALLI,MG,LI,NaN,NaN,NaN,NaN
1349,298.15,3000.000,NaN,ALLI,MG,LI,NaN,NaN,NaN,NaN


In [85]:
df_final[t_columns]

,T**(-9.0),T**(-7.0),T**(-5.0),T**(-3.0),T**(-2.0),T**(-1.0),T**0,T**1.0,T**2.0,T**3.0,T**4.0,T**7.0
1327,NaN,NaN,NaN,NaN,NaN,157900.0,-10534.680,287.351094,0.000972,-2.787338e-06,NaN,NaN
1328,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN
1329,NaN,NaN,NaN,NaN,NaN,78950.0,41632.660,143.675547,0.000486,-1.393669e-06,NaN,NaN
1330,1.038192e+28,NaN,NaN,NaN,NaN,NaN,35869.815,204.716215,NaN,NaN,NaN,NaN
1344,NaN,NaN,NaN,NaN,NaN,78950.0,-2781.340,143.675547,0.000486,-1.393669e-06,NaN,NaN
1345,NaN,NaN,NaN,NaN,NaN,NaN,2486.000,NaN,NaN,NaN,NaN,NaN
1346,NaN,NaN,NaN,NaN,NaN,74092.0,1292.760,223.048446,0.018532,-5.764227e-06,NaN,NaN
1347,NaN,NaN,NaN,NaN,NaN,NaN,2486.000,NaN,NaN,NaN,NaN,NaN
1348,NaN,NaN,NaN,NaN,NaN,159994.0,-19751.817,217.637482,0.035467,-1.986982e-05,NaN,NaN
1349,NaN,NaN,NaN,NaN,NaN,NaN,-9168.000,NaN,NaN,NaN,NaN,NaN
